In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score

#from cm import plot_confusion_matrix

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [3]:
faults = pd.read_csv('../data/filtered_df.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
faults

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,1,2,990360,2015-02-21 11:34:34.000,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,2,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
3,3,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
4,4,7,990439,2015-02-21 11:40:52.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577226,608448,1248448,123899434,2020-03-06 13:12:43.000,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,1936,105355619,30.376851,-81.744953,2020-03-06 13:29:33.000
577227,608450,1248452,123901805,2020-03-06 13:42:48.000,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,True,93,1886,105351219,39.015694,-77.031157,2020-03-06 13:43:24.000
577228,608451,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
577229,608452,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000


In [5]:
faults[faults['EquipmentID']==1641]['ecuMake'].value_counts()

unknown    3171
PACCR      2198
BNDWS      1427
CMMNS       404
?MMNS         1
Name: ecuMake, dtype: int64

In [6]:
faults[['EventTimeStamp', 'LocationTimeStamp']] = faults[['EventTimeStamp', 'LocationTimeStamp']].apply(pd.to_datetime)

In [7]:
#faults.info()

In [8]:
# Create list to filter dataset to only vehicles that had a 5246 at any point
derate_id = list(faults[faults['spn'] == 5246]['EquipmentID'].unique())

In [9]:
# Filter using derate_id list, then sort by truck and EventTimeStamp
derate_faults = faults[faults['EquipmentID'].isin(derate_id)].sort_values(['EquipmentID', 'EventTimeStamp'])

In [10]:
derate_faults[derate_faults['EquipmentID']==1641]['ecuMake'].value_counts()

Series([], Name: ecuMake, dtype: int64)

In [11]:
derate_faults

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
35505,36763,68953,2661725,2015-05-28 13:31:41,Condition Exists Cruise Control Enable Switch,unknown,unknown,unknown,unknown,49,596,31,True,4,301,105431712,35.988148,-85.021527,2015-05-28 13:32:18
42972,44535,83119,2881935,2015-06-10 11:55:42,Incorrect Data J1939 Network #1 Primary Vehicl...,unknown,unknown,unknown,unknown,11,639,2,True,127,301,105431712,36.084490,-86.686388,2015-06-10 11:56:19
47046,48805,90793,3005443,2015-06-17 14:16:42,Condition Exists Cruise Control Enable Switch,unknown,unknown,unknown,unknown,49,596,31,True,5,301,105431712,36.136620,-85.535231,2015-06-17 14:17:18
48967,50785,94502,3066394,2015-06-21 11:28:14,High (Severity High) Engine Speed,unknown,unknown,unknown,unknown,49,190,0,True,126,301,105431712,35.587222,-86.443379,2015-06-21 11:28:50
59930,62182,113745,3348193,2015-07-07 05:56:08,Low (Severity Medium) Engine Coolant Level,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,111,18,True,1,301,105431712,36.066435,-86.435231,2015-07-08 05:12:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489510,511325,1030043,53253271,2018-06-26 10:28:32,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5742,10,True,1,305,105362919,38.192500,-85.707638,2018-06-26 10:29:03
489536,511352,1030107,53270404,2018-06-26 12:35:35,Error in System Engine Torque Limit Request - ...,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,1787,11,True,3,305,105362919,38.192037,-85.707824,2018-06-26 12:36:11
489537,511353,1030108,53270405,2018-06-26 12:35:35,Special Instructions Maximum Vehicle Speed Limit,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,74,14,True,3,305,105362919,38.192037,-85.707824,2018-06-26 12:36:11
489882,511734,1030864,53532183,2018-06-28 13:16:24,NaN,PC4__1284P4C_2*,6U13D13,MX,PCAR,0,5111,12,True,1,305,105362919,38.192314,-85.707916,2018-06-28 13:18:11


## Find codes within certain timeframe of 5246

In [12]:
# Initiate column that will be the time of the next 5246 (if any).
# For rows indicating a 5246, the time will equal that of the EventTimeStamp.
derate_faults['DerateTimeStamp'] = derate_faults[derate_faults['spn'] == 5246]['EventTimeStamp']

In [13]:
# Backfill up, so that all rows will now include the time of the next 5246
derate_faults['DerateTimeStamp'] = derate_faults.groupby('EquipmentID')['DerateTimeStamp'].bfill()

In [14]:
# Add a column that shows the TimeDelta until the next 5246.
derate_faults['TimeToDerate'] = derate_faults['DerateTimeStamp'] - derate_faults['EventTimeStamp']

In [15]:
derate_faults.head(1)

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,DerateTimeStamp,TimeToDerate
35505,36763,68953,2661725,2015-05-28 13:31:41,Condition Exists Cruise Control Enable Switch,unknown,unknown,unknown,unknown,49,596,31,True,4,301,105431712,35.988148,-85.021527,2015-05-28 13:32:18,2016-07-12 19:11:07,411 days 05:39:26


In [16]:
# Cleaning
derate_faults = derate_faults[['RecordID', 'EquipmentID', 'ESS_Id', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 
                         'ecuMake', 'ecuSource', 'MCTNumber', 'eventDescription', 
                         'spn', 'fmi', 'activeTransitionCount', 'EventTimeStamp', 'DerateTimeStamp', 
                         'TimeToDerate', 'Latitude', 'Longitude', 'LocationTimeStamp']]

In [17]:
#derate_faults.to_csv('../data/derate_faults.csv')

## Add full dataset back in

In [18]:
derate_faults = faults.merge(derate_faults[['RecordID', 'DerateTimeStamp', 'TimeToDerate']], on='RecordID', how='left')

In [19]:
derate_faults[derate_faults['EquipmentID']==1641]['ecuMake'].value_counts()

unknown    3171
PACCR      2198
BNDWS      1427
CMMNS       404
?MMNS         1
Name: ecuMake, dtype: int64

## Count number of times each fault happens with n-day windows

In [20]:
# Sort to prep for desired groups and rolling windows
count_df = derate_faults.reset_index().set_index('EventTimeStamp').sort_values([
    'EquipmentID', 'spn', 'fmi', 'EventTimeStamp'], ascending = [True, True, True, True])

In [21]:
#count_df

In [22]:
# Create groupby, then rolling window to get # of fault occurrences within last day. Will repeat with added days.
count_df_1 = count_df.groupby([
    'EquipmentID', 'spn', 'fmi'])['index'].rolling('1D').count().reset_index().rename(columns={'index':'fault_count1'})

In [23]:
count_df_1

,EquipmentID,spn,fmi,EventTimeStamp,fault_count1
0,301,91,1,2018-09-27 04:43:26,1.0
1,301,91,4,2018-09-27 04:43:13,1.0
2,301,91,8,2018-09-27 04:43:13,1.0
3,301,91,10,2018-08-21 06:30:07,1.0
4,301,91,10,2018-09-27 04:41:35,1.0
...,...,...,...,...,...
577226,310,5394,7,2018-05-22 17:27:54,1.0
577227,R1762,1761,17,2015-02-24 13:45:06,1.0
577228,R1762,1761,18,2015-02-24 15:31:17,1.0
577229,R1762,5848,9,2015-02-26 13:12:11,1.0


In [24]:
# Repeat with 7 days
count_df_7 = count_df.groupby([
    'EquipmentID', 'spn', 'fmi'])['index'].rolling('7D').count().reset_index().rename(columns={'index':'fault_count7'})

In [25]:
# Repeat with 30 days
count_df_30 = count_df.groupby([
    'EquipmentID', 'spn', 'fmi'])['index'].rolling('30D').count().reset_index().rename(columns={'index':'fault_count30'})

In [26]:
# Confirm sort of derate_faults, for successful merge. Then reset index, for merge.
derate_faults = derate_faults.sort_values([
    'EquipmentID', 'spn', 'fmi', 'EventTimeStamp'], ascending = [True, True, True, True]).reset_index()

In [27]:
#derate_faults

In [28]:
# Sort count_dfs to match derate_faults
count_df_1 = count_df_1.sort_values([
    'EquipmentID', 'spn', 'fmi', 'EventTimeStamp', 'fault_count1'], ascending = [True, True, True, True, True])
count_df_7 = count_df_7.sort_values([
    'EquipmentID', 'spn', 'fmi', 'EventTimeStamp', 'fault_count7'], ascending = [True, True, True, True, True])
count_df_30 = count_df_30.sort_values([
    'EquipmentID', 'spn', 'fmi', 'EventTimeStamp', 'fault_count30'], ascending = [True, True, True, True, True])

In [29]:
count_df_1

,EquipmentID,spn,fmi,EventTimeStamp,fault_count1
0,301,91,1,2018-09-27 04:43:26,1.0
1,301,91,4,2018-09-27 04:43:13,1.0
2,301,91,8,2018-09-27 04:43:13,1.0
3,301,91,10,2018-08-21 06:30:07,1.0
4,301,91,10,2018-09-27 04:41:35,1.0
...,...,...,...,...,...
577226,310,5394,7,2018-05-22 17:27:54,1.0
577227,R1762,1761,17,2015-02-24 13:45:06,1.0
577228,R1762,1761,18,2015-02-24 15:31:17,1.0
577229,R1762,5848,9,2015-02-26 13:12:11,1.0


In [30]:
#Merge count_df's into derate_df
derate_df = pd.concat([derate_faults, count_df_1[['fault_count1']],
                      count_df_7[['fault_count7']], count_df_30[['fault_count30']]], axis=1).set_index('index')

In [31]:
derate_df.head()

,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,DerateTimeStamp,TimeToDerate,fault_count1,fault_count7,fault_count30
index,,,,,,,,,,,,,,,,,,,,,,,,
501916,525433,1060989,63024840,2018-09-27 04:43:26,Low (Severity High) Accelerator Pedal Position 1,unknown,unknown,unknown,unknown,0,91,1,True,1,301,105351510,35.925185,-86.435231,2018-09-27 04:44:03,NaT,NaT,1.0,1.0,1.0
501915,525432,1060988,63024835,2018-09-27 04:43:13,Low Voltage (Accelerator Pedal Position 1),unknown,unknown,unknown,unknown,0,91,4,True,2,301,105351510,35.921481,-86.436435,2018-09-27 04:43:49,NaT,NaT,1.0,1.0,1.0
501914,525431,1060987,63024834,2018-09-27 04:43:13,Abnormal Frequency Accelerator Pedal Position 1,unknown,unknown,unknown,unknown,0,91,8,True,1,301,105351510,35.921481,-86.436435,2018-09-27 04:43:49,NaT,NaT,1.0,1.0,1.0
496888,519722,1048711,59066105,2018-08-21 06:30:07,Abnormal Rate of Change Accelerator Pedal Posi...,unknown,unknown,unknown,unknown,0,91,10,True,1,301,105351510,35.747314,-86.340740,2018-08-21 06:30:44,NaT,NaT,1.0,1.0,1.0
501913,525430,1060986,63024744,2018-09-27 04:41:35,Abnormal Rate of Change Accelerator Pedal Posi...,unknown,unknown,unknown,unknown,0,91,10,True,2,301,105351510,35.902453,-86.450509,2018-09-27 04:42:12,NaT,NaT,1.0,1.0,1.0


In [32]:
derate_df = derate_df[['RecordID', 'EquipmentID', 'ESS_Id', 'ecuSoftwareVersion', 'ecuModel', 'ecuMake', 
           'spn', 'fmi', 'activeTransitionCount', 'EventTimeStamp', 'DerateTimeStamp', 'TimeToDerate', 
           'fault_count1', 'fault_count7', 'fault_count30']]

In [33]:
derate_df[derate_df['EquipmentID']==1641]['ecuMake'].value_counts()

unknown    3171
PACCR      2198
BNDWS      1427
CMMNS       404
?MMNS         1
Name: ecuMake, dtype: int64

In [34]:
# Test to determine whether to filter to only CMMNS.
# Keep for now - but we may remove it after running models.
#derate_df[derate_df['spn']==5246]['ecuMake'].value_counts()

In [35]:
#faults['ecuMake'].value_counts(normalize=True)

In [36]:
#Make a key of the spn + fmi
derate_df['SpnFmi'] = derate_df['spn'].astype('str')+'-'+derate_df['fmi'].astype('str')

### Load and Configure Vehicle Diagnosis Onboard Data

In [37]:
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [38]:
diagnostics

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [39]:
# Pivot to wide
diag_pivot = diagnostics.pivot_table('Value','FaultId','Name',aggfunc = 'first')

In [40]:
# Merge to derate_df
derate_diagnostics = derate_df.merge(diag_pivot, how = 'left', left_on = 'RecordID', right_on = 'FaultId')

In [41]:
derate_diagnostics

,RecordID,EquipmentID,ESS_Id,ecuSoftwareVersion,ecuModel,ecuMake,spn,fmi,activeTransitionCount,EventTimeStamp,DerateTimeStamp,TimeToDerate,fault_count1,fault_count7,fault_count30,SpnFmi,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1060989,301,63024840,unknown,unknown,unknown,91,1,1,2018-09-27 04:43:26,NaT,NaT,1.0,1.0,1.0,91-1,38,14.4275,False,0,370672.7,149,43,69.02,186.8,1539.5,8392.8,61.6,51549.741711124,8.215776,87.8,True,77,17407,False,NaN,58.45744,NaN,38,7.83
1,1060988,301,63024835,unknown,unknown,unknown,91,4,2,2018-09-27 04:43:13,NaT,NaT,1.0,1.0,1.0,91-4,38.8,14.4275,False,0,370672.5,149,46,67.28,187.3062,1442,8392.8,61.6,51549.741711124,8.783748,87.8,True,75.2,17407,False,NaN,54.73406,NaN,38.8,7.25
2,1060987,301,63024834,unknown,unknown,unknown,91,8,1,2018-09-27 04:43:13,NaT,NaT,1.0,1.0,1.0,91-8,38.8,14.4275,False,0,370672.5,149,46,67.28,187.3062,1442,8392.8,61.6,51549.741711124,8.783748,87.8,True,75.2,17407,False,NaN,54.73406,NaN,38.8,7.25
3,1048711,301,59066105,unknown,unknown,unknown,91,10,1,2018-08-21 06:30:07,NaT,NaT,1.0,1.0,1.0,91-10,54,14.355,False,0,367979.4,152.6,45,68.43999,167,1301.75,8293.7,20,51127.726858054,7.978021,87.8,True,89.6,17407,False,NaN,66.99158,NaN,68,6.38
4,1060986,301,63024744,unknown,unknown,unknown,91,10,2,2018-09-27 04:41:35,NaT,NaT,1.0,1.0,1.0,91-10,27.6,14.4275,False,0,370670.9,152.6,35,60.32,186.8,1288.5,8392.75,62.8,51549.609625098,5.877846,87.8,True,78.8,17407,False,NaN,66.57895,NaN,27.2,9.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577226,1016600,310,49722213,05317106*04133655*061416163421*09401361*G1*BDR*,6X1u13D1500000000,CMMNS,5394,7,1,2018-05-22 17:27:54,NaT,NaT,1.0,1.0,1.0,5394-7,99.2,14.355,False,64.6226,267594.5,183.2,93,38.28,217.4563,1316.375,7880.85,74.4,36081.93972242,19.23179,NaN,True,129.2,17407,False,NaN,64.21483,NaN,100,29.58
577227,4246,R1762,1048627,05317106*04119044*051914190353*09400015*G1*BDR*,6X1u13D1500000000,CMMNS,1761,17,43,2015-02-24 13:45:06,NaT,NaT,1.0,1.0,1.0,1761-17,0,14.065,True,66.48672,79796.12,185,83,42.34,222.6875,1333.25,1527.45,38,11368.512171794,14.72764,32,True,82.4,1023,False,NaN,65.96243,3276.75,47.2,16.82
577228,4428,R1762,1051023,05317106*04119044*051914190353*09400015*G1*BDR*,6X1u13D1500000000,CMMNS,1761,18,11,2015-02-24 15:31:17,NaT,NaT,1.0,1.0,1.0,1761-18,0,14.1375,True,66.48672,79913.24,179.6,53,42.34,222.6875,1352.875,1529.25,22,11390.042194032,9.166799,32,True,82.4,1023,False,NaN,66.83624,3276.75,67.2,8.99
577229,6439,R1762,1089561,05317106*04119044*051914190353*09400015*G1*BDR*,6X1u13D1500000000,CMMNS,5848,9,1,2015-02-26 13:12:11,NaT,NaT,1.0,1.0,1.0,5848-9,0,14.21,False,66.48672,80838.7,134.6,11,38.86,148.7188,595.75,1547.55,22.4,11527.675833124,0.6604322,32,True,84.2,17407,False,NaN,2.058292,3276.75,0,0.29


In [42]:
derate_diagnostics['EquipmentID'].value_counts()

1692    9968
1641    7386
1646    7086
1606    6982
1625    6735
        ... 
2170       1
2032       1
1483       1
2356       1
2380       1
Name: EquipmentID, Length: 1834, dtype: int64

In [43]:
derate_diagnostics[derate_diagnostics['EquipmentID']=="1641"]['ecuMake'].value_counts()

unknown    3171
PACCR      2198
BNDWS      1427
CMMNS       404
?MMNS         1
Name: ecuMake, dtype: int64

In [44]:
#derate_diagnostics.info()

In [45]:
#derate_diagnostics

In [46]:
#derate_diagnostics.to_csv('../data/derate_diagnostics.csv')

## Investigate Fault Codes Descriptions
#### Made judgment not to include them, due to highly imbalanced values, and redudant merge on SPN/FMI

In [47]:
fault_codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')

In [48]:
fault_codes.head(1)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...


In [49]:
fault_codes['PID'].value_counts()

Not Mapped    6755
190             15
157             14
91              12
110             12
94              12
105             11
103             11
102              9
175              8
441              8
98               8
100              7
132              7
443              7
352              7
97               7
174              7
111              7
137              7
21               6
412              6
129              6
411              6
46               6
173              6
156              6
159              6
177              6
101              6
84               6
372              6
109              5
149              5
16               5
108              5
96               5
27               5
135              5
15               5
168              5
171              5
82               4
131              4
51               4
351              4
52               4
167              4
155              4
17               4
191              3
127              3
99          

In [50]:
fault_codes['SID'].value_counts()

Not Mapped    5511
155            765
151            131
27              24
126             20
              ... 
128              1
216              1
145              1
372              1
275              1
Name: SID, Length: 163, dtype: int64

In [51]:
fault_codes['MID'].value_counts()

Not Mapped    5511
0              993
128            613
130              4
142              3
Name: MID, dtype: int64